# Imports

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
# Загрузка размеченных данных (пример)
goods = pd.read_csv('goods_all.csv')

# Загрузка данных с товарами для сравнения (пример)
goods_ours = pd.read_csv('datasets/napoleon_it_retail/itmo_meta.csv', index_col='Unnamed: 0')

In [3]:
goods = goods.drop('text', axis=1)
goods.head()

,file_name,text_clean,feature
0,0185c837-bf4a-43c3-985c-2e772651b5b0.jpg,CAXAPNECOK БЕЛЫЙ a ФАСОВАННЫЙ 1КГ ГОСТ Lis _...,1
1,5d880e3b-dbf3-4ae9-82b1-3429dc8e1bad.jpg,озпааниилллиилииЕ ДОБРЫЙ НАПИТОК Б А СИЛ ГАЗ ...,0
2,3d9fe58a-784e-4952-892c-8bd3e4323d99.jpg,a ae ADREMALINE BUSH НАПИТОК SHEPTET МАНГО 04...,1
3,ab2adfca-123c-4a04-b9a7-30599f1cb138.jpg,ча у Напиток ХУГАРДЕН БЕЛОЕ if 0. 44n _...,0
4,7a3f28b0-34e8-4f24-8e61-0412a94eb59e.jpg,ERATOR ИСТОЧНИК ПИТЬЕВАЯ ВОДА НЕГАЗ _x000C_,1


In [4]:
goods_ours.head()

,analogue_name
0,Вино RED WINE кр. п/сл. (Россия) 1L
1,Виски WILLIAM LAWSON'S купажированный алк.40% ...
2,Вода питьевая СВЯТОЙ ИСТОЧНИК природная негаз....
3,Квас 365 ДНЕЙ фильтрованный паст. ПЭТ (Россия)...
4,Кетчуп HEINZ томатный дой-пак (Россия) 320г


In [5]:
# Создаем функцию для обработки текста
def preprocess(text):
    text = text.lower()
    text = text.replace('ё', 'е')
    text = text.replace('ъ', 'ь')
    text = text.replace('й', 'и')
    text = re.sub('[^а-яА-яa-zA-Z0-9 ]', ' ', text)
    return text

In [6]:
# Применяет функцию к столбцу датасета goods
goods['text_clean'] = goods['text_clean'].apply(preprocess)

In [7]:
goods.head()

,file_name,text_clean,feature
0,0185c837-bf4a-43c3-985c-2e772651b5b0.jpg,caxapnecok белыи a фасованныи 1кг гост lis ...,1
1,5d880e3b-dbf3-4ae9-82b1-3429dc8e1bad.jpg,озпааниилллиилиие добрыи напиток б а сил газ ...,0
2,3d9fe58a-784e-4952-892c-8bd3e4323d99.jpg,a ae adremaline bush напиток sheptet манго 04...,1
3,ab2adfca-123c-4a04-b9a7-30599f1cb138.jpg,ча у напиток хугарден белое if 0 44n ...,0
4,7a3f28b0-34e8-4f24-8e61-0412a94eb59e.jpg,erator источник питьевая вода негаз x000c,1


In [8]:
# Применяем функцию к столбцу датасета goods_ours
goods_ours['analogue_name'] = goods_ours['analogue_name'].apply(preprocess)

In [9]:
goods_ours.head()

,analogue_name
0,вино red wine кр п сл россия 1l
1,виски william lawson s купажированныи алк 40 ...
2,вода питьевая святои источник природная негаз ...
3,квас 365 днеи фильтрованныи паст пэт россия ...
4,кетчуп heinz томатныи дои пак россия 320г


In [10]:
# Список наименований товаров из goods_ours для сравнения
comparison_data = goods_ours['analogue_name'].tolist()

In [11]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(goods['text_clean'], goods['feature'], test_size=0.2, random_state=73)

In [12]:
# Преобразование текстовых данных в числовые признаки с помощью TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [13]:
# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

MultinomialNB()

In [14]:
# Прогнозирование
y_pred = classifier.predict(X_test_vectorized)

In [15]:
# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность классификации на тестовой выборке: {accuracy * 100:.2f}%')

Точность классификации на тестовой выборке: 90.91%


In [16]:
# Применение классификатора к данным для сравнения
comparison_data_vectorized = vectorizer.transform(comparison_data)
predicted_labels = classifier.predict(comparison_data_vectorized)

In [17]:
# Создание DataFrame с результатами
results_df = pd.DataFrame({'Product1': comparison_data, 'IsAnalog': predicted_labels})

In [18]:
# Вывод результатов
for index, row in results_df.iterrows():
    if row["IsAnalog"] == 1:
        print(f"{row['Product1']} (предсказано как аналог)")
    else:
        print(f"{row['Product1']} (не предсказано как аналог)")

вино red wine кр  п сл   россия  1l (не предсказано как аналог)
виски william lawson s купажированныи алк 40   россия  0 7l (не предсказано как аналог)
вода питьевая святои источник природная негаз  пэт  россия  1 5l (предсказано как аналог)
квас 365 днеи фильтрованныи паст  пэт  россия  1 5l (не предсказано как аналог)
кетчуп heinz томатныи дои пак  россия  320г (предсказано как аналог)
кефир 365 днеи 2 5  пл без змж  россия  800г (предсказано как аналог)
конфеты merci ассорти  германия  250г (предсказано как аналог)
конфеты toffifee орешки в карамели  германия  125г (предсказано как аналог)
напиток ароматизированныи martini bianco виноградосодерж  из виноград  сырья бел  сл   италия  1l (предсказано как аналог)
напиток безалкогольныи adrenaline rush энергия энергетическии газ ж б  россия  0 449l (не предсказано как аналог)
напиток безалкогольныи drive me оригинальныи энерг  газ  ж б  россия  0 449l (предсказано как аналог)
напиток безалкогольныи добрыи кола с газ пэт  россия  1l (не 

In [19]:
results_df.to_csv('NB_results_df.csv', index=False)

In [20]:
results_df.head(20)

,Product1,IsAnalog
0,вино red wine кр п сл россия 1l,0
1,виски william lawson s купажированныи алк 40 ...,0
2,вода питьевая святои источник природная негаз ...,1
3,квас 365 днеи фильтрованныи паст пэт россия ...,0
4,кетчуп heinz томатныи дои пак россия 320г,1
5,кефир 365 днеи 2 5 пл без змж россия 800г,1
6,конфеты merci ассорти германия 250г,1
7,конфеты toffifee орешки в карамели германия ...,1
8,напиток ароматизированныи martini bianco виног...,1
9,напиток безалкогольныи adrenaline rush энергия...,0


In [22]:
import pickle

# Сохранение модели и векторизатора в файлы 'model.pkl' и 'vectorizer.pkl'
with open('tgbot/models/model.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

with open('tgbot/models/vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)